First, create the model. This must match the model used in the interactive training notebook.

In [4]:
import torch
import torchvision

CATEGORIES = ['orange_line']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

Next, load the saved model.  Enter the model path you used to save.

In [5]:
model.load_state_dict(torch.load('road_following_normalisation_orangeline_4.pth'))

<All keys matched successfully>

Convert and optimize the model using ``torch2trt`` for faster inference with TensorRT.  Please see the [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) readme for more details.

> This optimization process can take a couple minutes to complete. 

In [6]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

Save the optimized model using the cell below

In [7]:
torch.save(model_trt.state_dict(), 'trt_road_following_normalisation_orangeline_4.pth')

Load the optimized model by executing the cell below

In [8]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('trt_road_following_normalisation_orangeline_4.pth'))

<All keys matched successfully>

Create the racecar class

In [9]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

Create the camera class.

In [10]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)

Finally, execute the cell below to make the racecar move forward, steering the racecar based on the x value of the apex.

Here are some tips,

* If the car wobbles left and right,  lower the steering gain
* If the car misses turns,  raise the steering gain
* If the car tends right, make the steering bias more negative (in small increments like -0.05)
* If the car tends left, make the steering bias more postive (in small increments +0.05)

In [ ]:
#WITH WHEELS MOVING
from utils import preprocess
import numpy as np

STEERING_GAIN = 1.5 #0.95
STEERING_BIAS = -0.30

car.throttle = 0.2

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x=output[0]
    print(x)
    
    car.steering = x * STEERING_GAIN + STEERING_BIAS

0.5605
0.567
0.556
0.561
0.611
0.4036
0.361
0.3452
0.3281
0.3923
0.3953
0.1775
0.1735
0.1704
0.1569
0.0995
0.05112
0.03702
0.0846
0.0958
0.1631
0.1813
0.23
0.2034
0.1742
0.1183
0.14
0.0631
-0.2157
-0.355
-0.4983
-0.5273
-0.4243
-0.332
-0.1096
-0.01994
0.04642
0.145
0.1785
0.1788
0.1642
0.1859
0.2047
0.1982
0.1996
0.2026
0.2053
0.231
0.2703
0.2937
0.3076
0.329
0.348
0.357
0.375
0.3936
0.3992
0.4172
0.4304
0.4622
0.464
0.4583
0.4368
0.4192
0.4224
0.3892
0.3938
0.392
0.3586
0.3352
0.318
0.3125
0.3096
0.3098
0.3047
0.3037
0.2832
0.2778
0.2795
0.2805
0.2842
0.2864
0.2903
0.2559
0.2468
0.269
0.2605
0.2532
0.2593
0.2554
0.2393
0.2235
0.2014
0.1788
0.1631
0.1511
0.1456
0.1366
0.1194
0.0991
0.07764
0.0626
0.02658
0.002213
-0.01407
-0.03247
-0.08936
-0.11414
-0.1315
-0.1631
-0.1826
-0.1986
-0.2184
-0.2512
-0.269
-0.2861
-0.306
-0.3198
-0.338
-0.3562
-0.3713
-0.3826
-0.3943
-0.408
-0.4111
-0.4314
-0.4475
-0.471
-0.4941
-0.5146
-0.544
-0.5522
-0.5615
-0.5625
-0.565
-0.552
-0.544
-0.5547
-0.567
-0.

In [ ]:
#1CAT WITHOUT WHEELS MOVING 
from utils import preprocess
import numpy as np
import time
import matplotlib.pyplot as plt
global dataset

STEERING_GAIN = 0.95
STEERING_BIAS = -0.30

#car.throttle = 0.0

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model(image).detach().cpu().numpy().flatten()
    outputtest=np.exp(output)/sum(np.exp(output))
    prediction_ = model(image)
    prediction_prob=torch.nn.functional.softmax(prediction_,dim=1)
    print("pred",prediction_.detach().cpu().numpy().flatten())
    print("prob",prediction_prob.detach().cpu().numpy().flatten())
    print("\n")
    print(outputtest)
    category_number = outputtest.argmax()
    print(category_number)
    print("\n")
    
    time.sleep(1)
    #x=output[0]
    #car.steering = x * STEERING_GAIN + STEERING_BIAS

In [ ]:
#2CAT WITHOUT WHEELS MOVING 
from utils import preprocess
import numpy as np
import time
import matplotlib.pyplot as plt
global dataset

STEERING_GAIN = 0.95
STEERING_BIAS = -0.30

#car.throttle = 0.0

while True:
    image = camera.read()
    image = preprocess(image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    prediction_ = model_trt(image)
    prediction_prob=torch.nn.functional.softmax(prediction_,dim=1)
    print("pred X_tennis",prediction_.detach().cpu().numpy().flatten()[0])
    #print("prob X_tennis",prediction_prob.detach().cpu().numpy().flatten()[0])
    print("pred X_staple",prediction_.detach().cpu().numpy().flatten()[2])
    #print("prob X_staple",prediction_prob.detach().cpu().numpy().flatten()[2])
    print("\n")

    #time.sleep(1)
    #x=output[0]
    #car.steering = x * STEERING_GAIN + STEERING_BIAS